Podaci nisu podijeljeni na train/valid/test

In [102]:
import re
import spacy
import pandas as pd
from nltk.corpus import stopwords
from collections import Counter
nlp = spacy.load(
    "en_core_web_lg"
)
from sklearn.model_selection import train_test_split
df = pd.read_csv("Task 10/public_data/train_all_tasks.csv")

Svaki redak uzorka sadržava:

In [103]:
print(df.columns)

Index(['rewire_id', 'text', 'label_sexist', 'label_category', 'label_vector'], dtype='object')


Broj redaka:

In [104]:
print(df.shape[0])

14000


Radimo s nebalansiranim skupom podataka

Broj redaka iz klase seksistički:

In [105]:
print(len(df[df['label_sexist'] == 'sexist']))

3398


Broj redaka iz klase neseksistički:

In [106]:
print(len(df[df['label_sexist'] == 'not sexist']))

10602


Nema Nan podataka:

In [107]:
print(df.isnull().values.any())

False


In [108]:
najvise = 0
seksisti = []
neseksisti = []
prosrijeci = [0, 0]
djelitelj = [0, 0]
for i in df.sample(14000).itertuples():
    doc = nlp(i.text.lower())
    najvise = max(najvise, len(doc))
    br_rijeci = len(doc)
    tret = [token.lemma_
         for token in doc
         if not token.is_stop and not token.is_punct and token.is_alpha]
    if i.label_sexist == 'sexist':
        prosrijeci[0] += br_rijeci
        djelitelj[0] += 1
        for j in tret:
            seksisti.append(j)
    else:
        prosrijeci[1] += br_rijeci
        djelitelj[1] += 1
        for j in tret:
            neseksisti.append(j)

Najčešće riječi za klasu seksisti:

In [109]:
word_freq = Counter(seksisti)
common_words = word_freq.most_common(5)
for i in common_words:
    print(i)

('woman', 1726)
('man', 729)
('like', 502)
('bitch', 498)
('fuck', 359)


Najčešće riječi za klasu ne seksisti:

In [110]:
word_freq = Counter(neseksisti)
common_words = word_freq.most_common(5)
for i in common_words:
    print(i)

('woman', 2626)
('url', 1516)
('like', 1300)
('girl', 1228)
('man', 1033)


Broj prosječnih riječi: (tokena)
[seksisti, neseksisti]

In [111]:
print(najvise)
prosrijeci[0] /= djelitelj[0]
prosrijeci[1] /= djelitelj[1]
print(prosrijeci)

86
[29.620364920541494, 27.688077721184683]


Razdvojimo na train/valid/test:

In [144]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

lista_tekst = df["text"]
lista_label = df["label_sexist"]

for train_index, test_index in sss.split(lista_tekst, lista_label):
    X_train, X_test = lista_tekst[train_index], lista_tekst[test_index]
    y_train, y_test = lista_label[train_index], lista_label[test_index]
    d_train = {"text": X_train, "label": y_train}
    df_train = pd.DataFrame(d_train)
    d_test = {"text": X_test, "label": y_test}
    df_test = pd.DataFrame(d_test)

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.125, random_state=42)

lista_tekst = np.array(df_train["text"])
lista_label = np.array(df_train["label"])

for train_index, valid_index in sss.split(lista_tekst, lista_label):
    X_train, X_valid = lista_tekst[train_index], lista_tekst[valid_index]
    y_train, y_valid = lista_label[train_index], lista_label[valid_index]
    d_train = {"text": X_train, "label": y_train}
    df_train = pd.DataFrame(d_train)
    d_valid = {"text": X_valid, "label": y_valid}
    df_valid = pd.DataFrame(d_valid)

print(len(df_test), len(df_valid), len(df_train))


2800 1400 9800


Pogledajmo [URL], [USER], emoji:

Za [URL]:

In [152]:
#nije najtočnije ali radi skoro
urlovi = [0, 0]
sexist_c = len(df_train[df_train['label'] == 'sexist'])
nsexist_c = len(df_train[df_train['label'] == 'not sexist'])
for i in df_train.itertuples():
    doc = nlp(i.text)
    tret = [token
         for token in doc
         if "URL" == str(token)]
    kolko = len(tret)
    if i.label == 'sexist':
        urlovi[0] += kolko
    else:
        urlovi[1] += kolko
print(urlovi)
print(urlovi[0] / sexist_c, urlovi[0] / nsexist_c)

[134, 1051]
0.056349873843566024 0.018054432767448128


Za [USER]:

In [154]:
#nije najtočnije ali radi skoro
useri = [0, 0]
for i in df_train.itertuples():
    doc = nlp(i.text)
    tret = [token
         for token in doc
         if "USER" == str(token)]
    kolko = len(tret)
    if i.label == 'sexist':
        useri[0] += kolko
    else:
        useri[1] += kolko
print(useri)
print(useri[0] / sexist_c, useri[1] / nsexist_c)

[97, 523]
0.0407905803195963 0.07046618162220426


Za emoji:

In [159]:
import re
a_list = ['🤔 🙈 me así, bla es se 😌 ds 💕👭👙']
re.findall(r'[^\w\s,]', a_list[0])

['🤔', '🙈', '😌', '💕', '👭', '👙']

In [161]:
emojiji = [0, 0]
for i in df_train.itertuples():
    doc = nlp(i.text)
    kolko = len(re.findall(r'[^\w\s,]', str(doc)[0]))
    if i.label == 'sexist':
        emojiji[0] += kolko
    else:
        emojiji[1] += kolko
print(emojiji)
print(emojiji[0] / sexist_c, emojiji[1] / nsexist_c)

[118, 614]
0.0496215306980656 0.08272702775532202
